In [9]:
from icecream import ic
import pandas as pd
from dateutil.relativedelta import relativedelta
# from dhs_pca_functions import *
from pandarallel import pandarallel
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
# Initialization
pandarallel.initialize()

INFO: Pandarallel will run on 24 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [10]:
# folder: path = "/mnt/datadisk/data/surveys/DHS_final_raw_data/"
# work_dir = "/mnt/datadisk/data/Projects/water/pickles/"
input_dir = "/mnt/datadisk/data/Projects/water/inputs/"
# min_version = 3
overwrite_pqt = True
dataset_type = 'HR'  #, 'BR', 'CR', 'HW', 'IR', 'KR', 'MR', 'PR', 'AR'] ??['AN', 'FC, 'FP', 'HW', 'PV', 'SC'] < 25 surveys
add_str = '_w_NANs'
group_by_col = 'adm2_gaul'

in_f = f"{input_dir}grouped_df_V3_{dataset_type}_{group_by_col}{add_str}.pkl"
# categorical_in_f = f"{input_dir}grouped_df_V3_{dataset_type}_{group_by_col}{add_str}.pkl"
fewsNet_f = f"{input_dir}fewsnet_gaul_adm2.csv"

out_f = f"{input_dir}grouped_df_V3_{dataset_type}_{group_by_col}_joined_with_ipc{add_str}.pkl"

In [11]:
df = pd.read_pickle(in_f)
df

,GEID_init,adm2_gaul,adm1_gaul,adm0_gaul,year,month,date,number of household members: mean,number of household members: median,number of household members: std,...,has television: NaN,has motorcycle/scooter: no,has motorcycle/scooter: yes,has motorcycle/scooter: NaN,water usually treated by: use water filter: no,water usually treated by: use water filter: yes,water usually treated by: use water filter: NaN,"owns livestock, herds or farm animals: no","owns livestock, herds or farm animals: yes","owns livestock, herds or farm animals: NaN"
0,AOGE52FL,Ambaca,Kuanza Norte,Angola,2007,3,2007-03-06,5.160000,5.0,1.907878,...,0.0,0.920000,0.080000,0.0,NaN,NaN,1.000000,NaN,NaN,1.0
1,AOGE52FL,Amboim,Cuanza Sul,Angola,2006,12,2006-12-11,5.280000,5.0,2.282542,...,0.0,0.600000,0.400000,0.0,NaN,NaN,1.000000,NaN,NaN,1.0
2,AOGE52FL,Andulo,Bie,Angola,2007,1,2007-01-08,4.000000,4.0,1.632993,...,0.0,0.920000,0.080000,0.0,NaN,NaN,1.000000,NaN,NaN,1.0
3,AOGE52FL,Baia Farta,Benguela,Angola,2006,11,2006-11-27,5.208333,5.0,2.283764,...,0.0,0.791667,0.208333,0.0,NaN,NaN,1.000000,NaN,NaN,1.0
4,AOGE52FL,Balombo,Benguela,Angola,2006,12,2006-12-07,4.210526,4.0,1.618605,...,0.0,0.947368,0.052632,0.0,NaN,NaN,1.000000,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15247,ZWGE72FL,Umzingwane,Matabeleland South,Zimbabwe,2015,7,2015-07-10,4.642857,4.0,2.865006,...,0.0,0.976190,0.023810,0.0,0.97619,0.02381,0.000000,0.190476,0.809524,0.0
15248,ZWGE72FL,Uzumba-Maramba-Pfungwe,Mashonaland East,Zimbabwe,2015,10,2015-10-12,4.075949,4.0,2.080262,...,0.0,1.000000,0.000000,0.0,1.00000,0.00000,0.000000,0.151899,0.848101,0.0
15249,ZWGE72FL,Zaka,Masvingo,Zimbabwe,2015,10,2015-10-19,4.366337,4.0,2.415462,...,0.0,1.000000,0.000000,0.0,1.00000,0.00000,0.000000,0.069307,0.930693,0.0
15250,ZWGE72FL,Zvimba,Mashonaland West,Zimbabwe,2015,11,2015-11-25,4.546154,4.0,2.385718,...,0.0,0.976923,0.023077,0.0,1.00000,0.00000,0.007692,0.361538,0.638462,0.0


In [12]:
cols = []
for c in df.columns:
    if not ': ' in c:
        cols.append('Meta; ' + c)
    elif 'percentage of valid answers' in c:
        cols.append('Meta; ' + c)
    elif ': NaN' in c:
        cols.append('Meta; ' + c)
    elif any([x in c for x in ['mean', 'median', 'std', 'skewness', 'kurtosis']]):
        cols.append('DHS Num; ' + c)
    else:
        cols.append('DHS Cat; ' + c)
df.columns = cols
df
    

,Meta; GEID_init,Meta; adm2_gaul,Meta; adm1_gaul,Meta; adm0_gaul,Meta; year,Meta; month,Meta; date,DHS Num; number of household members: mean,DHS Num; number of household members: median,DHS Num; number of household members: std,...,Meta; has television: NaN,DHS Cat; has motorcycle/scooter: no,DHS Cat; has motorcycle/scooter: yes,Meta; has motorcycle/scooter: NaN,DHS Cat; water usually treated by: use water filter: no,DHS Cat; water usually treated by: use water filter: yes,Meta; water usually treated by: use water filter: NaN,"DHS Cat; owns livestock, herds or farm animals: no","DHS Cat; owns livestock, herds or farm animals: yes","Meta; owns livestock, herds or farm animals: NaN"
0,AOGE52FL,Ambaca,Kuanza Norte,Angola,2007,3,2007-03-06,5.160000,5.0,1.907878,...,0.0,0.920000,0.080000,0.0,NaN,NaN,1.000000,NaN,NaN,1.0
1,AOGE52FL,Amboim,Cuanza Sul,Angola,2006,12,2006-12-11,5.280000,5.0,2.282542,...,0.0,0.600000,0.400000,0.0,NaN,NaN,1.000000,NaN,NaN,1.0
2,AOGE52FL,Andulo,Bie,Angola,2007,1,2007-01-08,4.000000,4.0,1.632993,...,0.0,0.920000,0.080000,0.0,NaN,NaN,1.000000,NaN,NaN,1.0
3,AOGE52FL,Baia Farta,Benguela,Angola,2006,11,2006-11-27,5.208333,5.0,2.283764,...,0.0,0.791667,0.208333,0.0,NaN,NaN,1.000000,NaN,NaN,1.0
4,AOGE52FL,Balombo,Benguela,Angola,2006,12,2006-12-07,4.210526,4.0,1.618605,...,0.0,0.947368,0.052632,0.0,NaN,NaN,1.000000,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15247,ZWGE72FL,Umzingwane,Matabeleland South,Zimbabwe,2015,7,2015-07-10,4.642857,4.0,2.865006,...,0.0,0.976190,0.023810,0.0,0.97619,0.02381,0.000000,0.190476,0.809524,0.0
15248,ZWGE72FL,Uzumba-Maramba-Pfungwe,Mashonaland East,Zimbabwe,2015,10,2015-10-12,4.075949,4.0,2.080262,...,0.0,1.000000,0.000000,0.0,1.00000,0.00000,0.000000,0.151899,0.848101,0.0
15249,ZWGE72FL,Zaka,Masvingo,Zimbabwe,2015,10,2015-10-19,4.366337,4.0,2.415462,...,0.0,1.000000,0.000000,0.0,1.00000,0.00000,0.000000,0.069307,0.930693,0.0
15250,ZWGE72FL,Zvimba,Mashonaland West,Zimbabwe,2015,11,2015-11-25,4.546154,4.0,2.385718,...,0.0,0.976923,0.023077,0.0,1.00000,0.00000,0.007692,0.361538,0.638462,0.0


In [13]:
food_security_df = pd.read_csv(fewsNet_f)
food_security_df.head()
ic(food_security_df.columns.to_list())
food_security_df = food_security_df[['adm2_name', 'month', 'year', 'CS', 'HA0']]
food_security_df['HA0'] = food_security_df['HA0'].fillna(0)
food_security_df['IPC + food help'] = food_security_df['CS'] + food_security_df['HA0']
food_security_df.drop(columns=['HA0'], inplace=True)
food_security_df.columns = ['adm2_name', 'month', 'year', 'IPC', 'IPC + food help']
print(len(food_security_df))
food_security_df.sample(10)

/tmp/ipykernel_1217541/2063534321.py:1: DtypeWarning: Columns (26,27,28,29,30,31,32,33,34,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  food_security_df = pd.read_csv(fewsNet_f)
ic| food_security_df.columns.to_list(): ['OBJECTID',
                                         'adm2_code',
                                         'adm2_name',
                                         'str2_year',
                                         'exp2_year',
                                         'adm1_code',
                                         'adm1_name',
                                         'status',
                                         'disp_area',
                                         'adm0_code',
                                         'adm0_name',
                                         'shape_leng',
                                         'FID_1',
                                         'Shape_Le_1',
                                   

104386


,adm2_name,month,year,IPC,IPC + food help
9297,Bali,4,2011,3.0,3.0
59657,Liwale,1,2010,1.0,1.0
74962,As Sayyani,6,2016,3.0,3.0
85281,Kulbus,6,2020,2.0,2.0
102997,Lubumbashi,10,2020,1.0,1.0
101173,Mchinji,2,2019,1.0,2.0
3336,Shendam,1,2010,1.0,1.0
39818,Gagal,6,2017,1.0,1.0
46814,Ihiala,2,2019,1.0,1.0
58264,Kolokuma/Opokuma,2,2022,2.0,2.0


In [14]:
def aggregate_ipc(row, food_security_df, start_delta, end_delta, y_m_indicator):
    # Create datetime objects for the start and end dates
    in_date = row['Meta; date']
    if y_m_indicator == 'm':
        start_date = in_date + relativedelta(months=start_delta)
        end_date = in_date + relativedelta(months=end_delta)
    elif y_m_indicator == 'y':
        start_date = in_date + relativedelta(years=start_delta)
        end_date = in_date + relativedelta(years=end_delta)
    else:
        raise ValueError(f"y_m_indicator should be 'm' or 'y', not {y_m_indicator}")
    # Filter the data for the given time span and adm2_gaul
    df_time_span = food_security_df[(food_security_df['date'] >= start_date) & (food_security_df['date'] < end_date) & (food_security_df['adm2_name'] == row['Meta; adm2_gaul'])]


    row[f'FS; IPC: {start_delta}-{end_delta}{y_m_indicator}: mean'] = df_time_span['IPC'].mean()
    row[f'FS; IPC: {start_delta}-{end_delta}{y_m_indicator}: max'] = df_time_span['IPC'].max()
    row[f'FS; IPC: {start_delta}-{end_delta}{y_m_indicator}: min'] = df_time_span['IPC'].min()
    row[f'FS; IPC: {start_delta}-{end_delta}{y_m_indicator}: median'] = round(df_time_span['IPC'].median(), 0)

    row[f'FS; IPC + FH: {start_delta}-{end_delta}{y_m_indicator}: mean'] = df_time_span['IPC + food help'].mean()
    row[f'FS; IPC + FH: {start_delta}-{end_delta}{y_m_indicator}: max'] = df_time_span['IPC + food help'].max()
    row[f'FS; IPC + FH: {start_delta}-{end_delta}{y_m_indicator}: min'] = df_time_span['IPC + food help'].min()
    row[f'FS; IPC + FH: {start_delta}-{end_delta}{y_m_indicator}: median'] = round(df_time_span['IPC + food help'].median(), 0)

    return row

# Convert year and month to datetime
food_security_df['date'] = pd.to_datetime(food_security_df[['year', 'month']].assign(day=1))
print(food_security_df.shape)
print(df.shape)
print('Meta; date' in df.columns)

for start_delta, end_delta, m_y_ind in [(0, 5, 'm'), (6, 12, 'm'), (1, 2, 'y'), (2, 4, 'y'), (4, 6, 'y'), (6, 7, 'y'), (6, 10, 'y'), (10, 15, 'y'), (15, 20, 'y'), (20, 25, 'y'),
                                        (0, 1, 'y'), (0, 2, 'y'), (2, 6, 'y'), (6, 12, 'y'), (12, 20, 'y')]:
    df = df.parallel_apply(aggregate_ipc, axis=1, food_security_df=food_security_df, start_delta=start_delta, 
                                            end_delta=end_delta, y_m_indicator=m_y_ind)

df[df['FS; IPC + FH: 15-20y: mean'].notna()]

(104386, 6)
(15252, 1477)
True


,Meta; GEID_init,Meta; adm2_gaul,Meta; adm1_gaul,Meta; adm0_gaul,Meta; year,Meta; month,Meta; date,DHS Num; number of household members: mean,DHS Num; number of household members: median,DHS Num; number of household members: std,...,FS; IPC + FH: 6-12y: min,FS; IPC + FH: 6-12y: median,FS; IPC: 12-20y: mean,FS; IPC: 12-20y: max,FS; IPC: 12-20y: min,FS; IPC: 12-20y: median,FS; IPC + FH: 12-20y: mean,FS; IPC + FH: 12-20y: max,FS; IPC + FH: 12-20y: min,FS; IPC + FH: 12-20y: median
32,AOGE52FL,Dala,Lunda Sul,Angola,2006,11,2006-11-23 00:00:00,3.880000,4.0,1.855622,...,1.0,1.0,1.500000,2.0,1.0,2.0,1.500000,2.0,1.0,2.0
287,BFGE32FL,Bale,Boucle Du Mouhoun,Burkina Faso,1999,2,1999-02-18 00:00:00,4.411765,4.0,2.425356,...,1.0,2.0,1.371795,4.0,1.0,1.0,1.461538,4.0,1.0,1.0
288,BFGE32FL,Bam,Centre-nord,Burkina Faso,1998,12,1998-12-09 12:00:00,7.105634,6.0,3.945925,...,1.0,2.0,1.076923,2.0,1.0,1.0,1.076923,2.0,1.0,1.0
289,BFGE32FL,Banwa,Boucle Du Mouhoun,Burkina Faso,1999,1,1999-01-22 00:00:00,7.176471,6.0,4.632125,...,1.0,1.0,1.000000,1.0,1.0,1.0,1.000000,1.0,1.0,1.0
290,BFGE32FL,Bazega,Centre-sud,Burkina Faso,1999,2,1999-02-07 00:00:00,8.756757,8.0,5.035460,...,1.0,1.0,1.000000,1.0,1.0,1.0,1.000000,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15122,ZWGE52FL,Umzingwane,Matabeleland South,Zimbabwe,2005,9,2005-09-23 00:00:00,5.146067,5.0,2.967822,...,1.0,1.0,2.571429,3.0,1.0,3.0,2.571429,3.0,1.0,3.0
15123,ZWGE52FL,Uzumba-Maramba-Pfungwe,Mashonaland East,Zimbabwe,2005,10,2005-10-19 00:00:00,3.847458,4.0,1.827129,...,1.0,1.0,2.692308,3.0,2.0,3.0,2.846154,3.0,2.0,3.0
15124,ZWGE52FL,Zaka,Masvingo,Zimbabwe,2005,11,2005-11-24 00:00:00,5.051095,5.0,2.638296,...,1.0,1.0,2.615385,3.0,2.0,3.0,2.692308,3.0,2.0,3.0
15125,ZWGE52FL,Zvimba,Mashonaland West,Zimbabwe,2005,12,2005-12-19 00:00:00,4.294872,4.0,2.275303,...,1.0,1.0,1.538462,3.0,1.0,1.0,1.692308,3.0,1.0,1.0


In [15]:
# save the dataframe
df.to_pickle(out_f)
df.to_csv(out_f.replace('.pkl', '.csv'), index=False)

In [16]:
df

,Meta; GEID_init,Meta; adm2_gaul,Meta; adm1_gaul,Meta; adm0_gaul,Meta; year,Meta; month,Meta; date,DHS Num; number of household members: mean,DHS Num; number of household members: median,DHS Num; number of household members: std,...,FS; IPC + FH: 6-12y: min,FS; IPC + FH: 6-12y: median,FS; IPC: 12-20y: mean,FS; IPC: 12-20y: max,FS; IPC: 12-20y: min,FS; IPC: 12-20y: median,FS; IPC + FH: 12-20y: mean,FS; IPC + FH: 12-20y: max,FS; IPC + FH: 12-20y: min,FS; IPC + FH: 12-20y: median
0,AOGE52FL,Ambaca,Kuanza Norte,Angola,2007,3,2007-03-06,5.160000,5.0,1.907878,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AOGE52FL,Amboim,Cuanza Sul,Angola,2006,12,2006-12-11,5.280000,5.0,2.282542,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AOGE52FL,Andulo,Bie,Angola,2007,1,2007-01-08,4.000000,4.0,1.632993,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AOGE52FL,Baia Farta,Benguela,Angola,2006,11,2006-11-27,5.208333,5.0,2.283764,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AOGE52FL,Balombo,Benguela,Angola,2006,12,2006-12-07,4.210526,4.0,1.618605,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15247,ZWGE72FL,Umzingwane,Matabeleland South,Zimbabwe,2015,7,2015-07-10,4.642857,4.0,2.865006,...,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15248,ZWGE72FL,Uzumba-Maramba-Pfungwe,Mashonaland East,Zimbabwe,2015,10,2015-10-12,4.075949,4.0,2.080262,...,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15249,ZWGE72FL,Zaka,Masvingo,Zimbabwe,2015,10,2015-10-19,4.366337,4.0,2.415462,...,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15250,ZWGE72FL,Zvimba,Mashonaland West,Zimbabwe,2015,11,2015-11-25,4.546154,4.0,2.385718,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
